In [1]:
# !pip install networkx
# !pip install scikit-image
# !pip install Pillow
# !pip install numpy
# !pip install pandas
# !pip install opencv-python
# !pip install plotly
# !pip install plotly-express
# !pip install dash
# !pip install jupyter-dash

import glob
import pandas as pd
import numpy as np
import networkx as nx
from itertools import combinations as comb
from collections import defaultdict
from scipy.stats import itemfreq
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage import feature
from PIL import Image as IMG
import numpy as np
import pandas as pd 
import operator
import cv2
import os 
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, no_update
import plotly.graph_objects as go
import random
import base64

In [2]:

def create_df(images_paths):
    features = pd.DataFrame()
    features['image']=image_paths
    features['dullness'] = [random.randint(0,10) for i in range(len(images_paths))]
    features['whiteness'] = [random.randint(10,100) for i in range(len(images_paths))]
    features['average_pixel_width'] = [random.randint(100,1000) for i in range(len(images_paths))]
    features['dominant_color'] = [random.randint(1000,10000) for i in range(len(images_paths))]
    features['classes'] = features['image'].apply(lambda x:x.split('\\')[-2])
    return features


        

In [3]:
pattern = "C:/Users/gprak/Downloads/youtubeVideos/datasets/natural_images/*/*.jpg"
image_paths = glob.glob(pattern)
df = create_df(image_paths)

In [4]:
df.head()

,image,dullness,whiteness,average_pixel_width,dominant_color,classes
0,C:/Users/gprak/Downloads/youtubeVideos/dataset...,1,85,949,4308,airplane
1,C:/Users/gprak/Downloads/youtubeVideos/dataset...,6,61,383,5010,airplane
2,C:/Users/gprak/Downloads/youtubeVideos/dataset...,9,52,210,9423,airplane
3,C:/Users/gprak/Downloads/youtubeVideos/dataset...,2,84,547,6062,airplane
4,C:/Users/gprak/Downloads/youtubeVideos/dataset...,8,90,447,8797,airplane


In [5]:
def create_3d_graph(features,class_wise,threshold):
    features = features[features['classes']==class_wise].reset_index()
    #print(features)
    features.drop(['classes'],axis=1,inplace=True)
    x=features['image']
    df = features
    df.index  = features['image'].tolist()
    df.drop(['image'],axis=1,inplace=True)
   
    data= df
    corr = data.T.corr()
    print(f"corr.shape = {corr.shape}")
    links = corr.stack().reset_index()
    links.columns = ['var1','var2','value']
    # Keep only correlation over a threshold and remove self correlation (cor(A,A)=1)
    links_filtered=links.loc[ (links['value'] > threshold) & (links['var1'] != links['var2']) ]
    df = links_filtered
    
    # Takes computation of weight sum over millions of edges into account 
    df.columns = ['Customer_A','Customer_B','Weight']
    df = df.groupby(
         df[['Customer_A', 'Customer_B']].apply(
             lambda row: tuple(sorted(row)),
             axis='columns'
         )
     ).sum().reset_index(drop=False).rename(columns={'index': 'Edges'})
    df[['Customer_A', 'Customer_B']] = pd.DataFrame(df.Edges.to_list())
    df.drop(columns=['Edges'], inplace=True)
    #G = nx.from_pandas_edgelist(links_filtered,'var1','var2')
    G = nx.from_pandas_edgelist(df, 'Customer_A', 'Customer_B', 'Weight')
    print(f"G.number_of_nodes() = {G.number_of_nodes()}, G.number_of_edges() = {G.number_of_edges()}")
    
    
    # spring layout is nothing but forces linked to distance between nodes such that every vertices 
    # comes into equilibrium similar to balancing act of vectors in physics when they came into their 
    # final state
    spring_3D = nx.spring_layout(G,dim=3,k=0.5)
    print(f"len(list(spring_3D.keys())) = {len(list(spring_3D.keys()))}")
    x_nodes = [spring_3D[key][0] for key in spring_3D.keys()]
    y_nodes = [spring_3D[key][1] for key in spring_3D.keys()]
    z_nodes = [spring_3D[key][2] for key in spring_3D.keys()]
    #print(f"x_nodes = {x_nodes}")
    res_df = pd.DataFrame()
    res_df["x"]=x_nodes
    res_df["y"]=y_nodes
    res_df["z"]=z_nodes
    res_df["img_url"]=x
    return res_df

# Using base64 encoding and decoding
def b64_image(image_filename):
    with open(image_filename, 'rb') as f:
        image = f.read()
    return 'data:image/png;base64,' + base64.b64encode(image).decode('utf-8')


In [6]:
res_df = create_3d_graph(df,'car',0.6)
res_df

corr.shape = (968, 968)
G.number_of_nodes() = 968, G.number_of_edges() = 460293
len(list(spring_3D.keys())) = 968


,x,y,z,img_url
0,0.291137,0.388450,0.063051,C:/Users/gprak/Downloads/youtubeVideos/dataset...
1,0.474756,0.058756,-0.100022,C:/Users/gprak/Downloads/youtubeVideos/dataset...
2,-0.087615,-0.409808,0.122552,C:/Users/gprak/Downloads/youtubeVideos/dataset...
3,-0.092640,0.334474,0.332948,C:/Users/gprak/Downloads/youtubeVideos/dataset...
4,-0.221748,-0.295683,-0.035881,C:/Users/gprak/Downloads/youtubeVideos/dataset...
...,...,...,...,...
963,0.079505,-0.331493,-0.941902,C:/Users/gprak/Downloads/youtubeVideos/dataset...
964,-0.036226,-0.493365,-1.000000,C:/Users/gprak/Downloads/youtubeVideos/dataset...
965,0.492510,-0.673351,-0.237381,C:/Users/gprak/Downloads/youtubeVideos/dataset...
966,-0.070718,-0.290617,-0.891758,C:/Users/gprak/Downloads/youtubeVideos/dataset...


In [7]:
def create_dash_app(df):
    fig = go.Figure(data=[
        go.Scatter3d(
            x=df['x'], 
            y=df['y'], 
            z=df['z'],
            mode='markers',
            #marker=dict(color=df['color'])
        )
    ])

    # turn off native plotly.js hover effects - make sure to use
    # hoverinfo="none" rather than "skip" which also halts events.
    fig.update_traces(hoverinfo="none", hovertemplate=None)
    fig.update_layout(
        scene = dict(
            xaxis = dict(range=[int(min(df['x'].tolist())),int(max(df['x'].tolist()))],),
                         yaxis = dict(range=[int(min(df['y'].tolist())),int(max(df['y'].tolist()))],),
                         zaxis = dict(range=[int(min(df['x'].tolist())),int(max(df['z'].tolist()))],),
        ),
        clickmode='event+select',
    )

    app = JupyterDash(__name__)

    server = app.server

    app.layout = html.Div([
        dcc.Graph(id="graph-basic-2", figure=fig, clear_on_unhover=True),
        dcc.Tooltip(id="graph-tooltip"),
    ])


    @app.callback(
        Output("graph-tooltip", "show"),
        Output("graph-tooltip", "bbox"),
        Output("graph-tooltip", "children"),
        Input("graph-basic-2", "hoverData"),
    )
    def display_hover(hoverData):
        if hoverData is None:
            return False, no_update, no_update

        # demo only shows the first point, but other points may also be available
        pt = hoverData["points"][0]
        bbox = pt["bbox"]
        num = pt["pointNumber"]

        df_row = df.iloc[num]
        img_src = df_row['img_url']
        #print(f"img_src={img_src}")
        #encoded_image = base64.b64encode(open(img_src, 'rb').read())

        
        children = [
            html.Div([
                html.Img(src=b64_image(img_src), style={"width": "100%"}),
            ], style={'width': '100px', 'white-space': 'normal'})
        ]

        return True, bbox, children
    return app
    

In [8]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
# apps = []
#df['classes']='single_class'
classes = df['classes'].unique()
@interact_manual(className=widgets.Combobox(options=tuple(list(classes)), value=classes[0]),threshold=(0,0.99,0.1))
def f(className,threshold):
    res_df = create_3d_graph(df,className,threshold)
    app = create_dash_app(res_df)
    print(f"app created successfully for Class = {className}")
    app.run_server(mode="external")
    



interactive(children=(Combobox(value='airplane', description='className', options=('airplane', 'car', 'cat', '…